In [1]:
from torchvision import datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

In [2]:
train_dataset = datasets.MNIST(root="./datasets/", train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root="./datasets/", train=False, download = True, transform=transforms.ToTensor())

In [3]:
imgs = torch.stack([img for img, _ in train_dataset], dim =0)
mean = imgs.view(1, -1).mean(dim =1)
std = imgs.view(1, -1).std(dim=1)


In [4]:
mnist_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

In [5]:
train_dataset = datasets.MNIST(root="./datasets", train=True, download = False, transform = mnist_transforms)
test_dataset = datasets.MNIST(root="./datasets", train=False, download = False, transform = mnist_transforms)

In [6]:
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset = train_dataset, lengths=[train_size, val_size])

In [7]:
train_loader = DataLoader(dataset = train_dataset, batch_size = 64, shuffle=True)
test_loader = DataLoader(dataset = test_dataset, batch_size = 64, shuffle=True)
val_loader = DataLoader(dataset = val_dataset, batch_size = 64, shuffle=True)

In [8]:
from torch import nn

class LeNet5V1(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature = nn.Sequential(
            #1
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),   # 28*28->32*32-->28*28
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),  # 14*14
            
            #2
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),  # 10*10
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),  # 5*5
            
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16*5*5, out_features=120),
            nn.Tanh(),
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=10),
        )
        
    def forward(self, x):
        return self.classifier(self.feature(x))

In [25]:
def fsgm(image, epsilon,data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon* sign_data_grad
    perturbed_data = torch.clamp(perturbed_image, 0,1)
    return perturbed_data

def denorm(batch, mean= mean, std = std):
    if isinstance(mean, list):
        mean = torch.tensor(mean).to(device)
    if isinstance(std, list):
        std = torch.tensor(std).to(device)

    return batch * std.view(1, -1, 1,1) + mean.view(1, -1, 1, 1)


In [27]:
from torchmetrics import Accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_lenet5v1 = LeNet5V1().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model_lenet5v1.parameters(), lr = 0.001)
accuracy = Accuracy(task="multiclass", num_classes =10)

In [109]:
from tqdm.notebook import tqdm
EPOCHS = 12

def train(model,device,  train_loader, epsilon):
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        X.requires_grad = True
        
        model.train()
        y_pred = model(X)
        
        loss = loss_fn(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
       
        data_grad = X.grad.data
        perturbed_data = fsgm(X, epsilon, data_grad)
    
        output = model(perturbed_data)
    
        loss_adv = loss_fn(output, y)

        optimizer.zero_grad()
        loss_adv.backward()
        optimizer.step()
            

def test(model,device,  test_loader, epsilon):
    model.eval()
    correct = 0
    adv_examples = []


    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        X.requires_grad = True
        
        output = model(X)
        
        init_pred = output.max(1, keepdim=True)[1][0]

        if init_pred.item() != y[0].item():
            continue

        loss = loss_fn(output, y)
        
        model.zero_grad()
        loss.backward()

        data_grad = X.grad.data
        
        perturbed_data = fsgm(X, epsilon, data_grad)

        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1][0]
        if final_pred.item() == y[0].item():
            correct += 1
            if len(adv_examples) < 5 and epsilon == 0:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
    
    final_acc = correct / float(len(test_loader))
    print(f"Epsilon: {epsilon}\t Test Accuracy= {final_acc}")
    
    return  final_acc, adv_examples
                
        
        
    

In [111]:
epsilons = [0, 0.05, 0.1, 0.15, 0.20, 0.25, 0.30]
device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model_lenet5v1

for epoch in tqdm(range(EPOCHS)):
    train(model,device, train_loader, epsilon=0.1)

accuracies = []
examples = []

for eps in epsilons:
    acc, ex = test(model,device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)


Epsilon: 0	 Test Accuracy= 0.9044585987261147
Epsilon: 0.05	 Test Accuracy= 0.9617834394904459
Epsilon: 0.1	 Test Accuracy= 0.8980891719745223
Epsilon: 0.15	 Test Accuracy= 0.8980891719745223
Epsilon: 0.2	 Test Accuracy= 0.9171974522292994
Epsilon: 0.25	 Test Accuracy= 0.910828025477707
Epsilon: 0.3	 Test Accuracy= 0.8853503184713376
